# Theano version of simple DNN version 7

***What this version done ***
1. Adjust dropout
2. Play MFCC and FBank
3. Data collect

***Todo***
- Add [RBM](http://deeplearning.net/tutorial/rbm.html)
- RBM http://blog.echen.me/2011/07/18/introduction-to-restricted-boltzmann-machines/
- RBM https://class.coursera.org/neuralnets-2012-001/lecture
- RBM https://github.com/lmjohns3/py-rbm/blob/master/lmj/rbm.py
- RBM http://imonad.com/rbm/restricted-boltzmann-machine/
2. ~~Add other method -- nesterov_momentum, relu, tanh~~
3. debug auto adjust since we just linearly divide by max_epoch for every epoch
4. [Add keyboardinterrupt exception then save the model](http://stackoverflow.com/questions/21120947/catching-keyboardinterrupt-in-python-during-program-shutdown)
5. Add unsupervised learning method http://www.csri.utoronto.ca/~hinton/absps/googlerectified.pdf
6. Change data to filter-bank
7. ~~fix X.dot(w)+b~~
8. ~~Check dropout method problem~~
9. Add maxout network
- ~~Add regularization term to every hidden layer~~
- Add regularization to cost function
- preprocess data with normalize and experiment the paper result       [tool](http://scikit-learn.org/stable/modules/preprocessing.html) 
> The open source Kaldi toolkit (Povey et al., 2011) was used to preprocess the data into log-
filter banks. A monophone system was trained to do a forced alignment and to get labels for
speech frames. Dropout neural networks were trained on windows of 21 consecutive frames
to predict the label of the central frame. No speaker dependent operations were performed.
***The inputs were mean centered and normalized to have unit variance***.

- What hinton to for their project include preprocessing http://www.cs.toronto.edu/~nitish/deepnet/
-------

***Tune variable***
1. Change hidden layer size
2. Change dropout ratio
3. Change activation function
4. [Weight initializate](http://deeplearning.net/tutorial/mlp.html#weight-initialization)
5. > We used probability of retention p = 0.8 in the input layers and 0.5 in the hidden layers.
Max-norm constraint with c = 4 was used in all the layers. A momentum of 0.95 with a
high learning rate of 0.1 was used. The learning rate was decayed as $\epsilon_0 \frac{1}{(1 + t/T)}$, from [Dropout: A Simple Way to Prevent Neural Networks from
Overfitting](http://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf)

***Knowledge need to fill the gap***
1. Numpy broadcast http://wiki.scipy.org/EricsBroadcastingDoc
2. Rewrite deep equation

***Theano need to read and understand***
- Scan (for loop in gpu) function http://deeplearning.net/software/theano/library/scan.html
- Plot weight http://deeplearning.net/tutorial/utilities.html#how-to-plot

***Question***
1. Why dont need a bias term b in the equation or even initialize?
2. ~~Once apply dropout, the result is strange~~ [quora](http://www.quora.com/How-would-you-implement-drop-out-in-a-deep-neural-network) [github](https://github.com/mdenil/dropout/blob/master/mlp.py#L261)
3. Softmax weight divide by 2 ??

***Github***
- https://gist.github.com/SnippyHolloW/8a0f820261926e2f41cc
- https://github.com/dnouri/nolearn/blob/master/nolearn/lasagne.py
- https://github.com/benanne/Lasagne/blob/master/lasagne/nonlinearities.py
- https://github.com/nitishsrivastava/deepnet/tree/master/deepnet

***Paper***
1. [Improving neural networks by preventing
co-adaptation of feature detectors](http://arxiv.org/pdf/1207.0580.pdf)
> Visible biases were initialized to zero and weights to random numbers sampled from a zeromean
normal distribution with standard deviation 0.01. The variance of each visible unit was
set to 1.0 and not learned. Learning was done by minimizing Contrastive Divergence. Momentum
was used to speed up learning. Momentum started at 0.5 and was increased linearly to 0.9
over 20 epochs. A learning rate of 0.001 on the average gradient was used (which was then
multiplied by 1-momentum). An L2 weight decay of 0.001 was used. The model was trained
for 100 epochs.

> The pretrained RBMs were used to initialize the weights in a neural network. The network
was then finetuned with dropout-backpropagation. Momentum was increased from 0.5 to 0.9
linearly over 10 epochs. A small constant learning rate of 1.0 was used (applied to the average
gradient on a minibatch). All other hyperparameters are the same as for MNIST dropout finetuning.
The model needs to be run for about 200 epochs to converge. The same network was
also finetuned with standard backpropagation using a smaller learning rate of 0.1, keeping all
other hyperparameters

2. [Dropout: A Simple Way to Prevent Neural Networks from
Overfitting](http://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf)
3. [Maxout Network](http://arxiv.org/pdf/1302.4389v4.pdf)

***References***
0. [Ipython Markdown](http://nbviewer.ipython.org/github/twistedhardware/mltutorial/blob/master/notebooks/IPython-Tutorial/2%20-%20Markdown%20%26%20LATEX.ipynb)
- Dropout, Hinton http://arxiv.org/pdf/1207.0580.pdf
- Dropout, Pratical guide http://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf
- Dropout, MaxOutnetwork http://arxiv.org/pdf/1302.4389v4.pdf
- [Several Deep](http://snippyhollow.github.io/blog/2014/08/09/so-you-wanna-try-deep-learning/) -- Dropout, you will see improvement only on large-enough networks (> 1000 units / layer, > 3-4 layers **[example](https://gist.github.com/SnippyHolloW/8a0f820261926e2f41cc)**
- Using cudamat implementation deep: expert github https://github.com/nitishsrivastava/deepnet
2. Tutorial http://neuralnetworksanddeeplearning.com/chap3.html
3. http://ieeexplore.ieee.org/xpl/articleDetails.jsp?tp=&arnumber=6296526&searchWithin%3Dp_Authors%3A.QT.Hinton%2C+G..QT.%26searchWithin%3Dp_Author_Ids%3A37270925500
4. http://ieeexplore.ieee.org/xpl/articleDetails.jsp?tp=&arnumber=5704567&searchWithin%3Dp_Authors%3A.QT.Hinton%2C+G..QT.%26searchWithin%3Dp_Author_Ids%3A37270925500%26sortType%3Ddesc_p_Publication_Year
5. Kaldi, acostic preprocess http://kaldi.sourceforge.net/

In [1]:
import theano
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
import numpy as np

from sklearn import cross_validation as cv
from sklearn import metrics
from sklearn import grid_search as gs
from sklearn.metrics import accuracy_score
# from sklearn.base import BaseEstimator
from itertools import repeat

from time import time

Using gpu device 0: GeForce GTX 980


## Data preprocess

In [9]:
### DataProcessing 

'''
[ INPUT  ] : model { wav, NFCC, ... }
[ OUTPUT ] : (Training_data, Validation_data, Testing_Data)

- Training_data : x, y { .astype(np.float32)
- Validation_data : x, y { .astype(np.float32)
- Testing_Data : ( x, id )

-- x format : 
---- NFCC : [ 39-vector ]
---- FBank : [ 69-vector ]
---- quadmfcc : [ 39*(1+1+1) - vector ] !!! Wrong implementation
---- hexamfcc : [ 39*(4+1+4) - vector ]
---- hexaFbank : [ 69*(4+1+4) - vector ]

-- y format :

---- Now we use 39-phonemes for all : [ 0 0 0 ... 1 .... 0 0 0 ]  as a number of 39
---- But answer we get from ./label is [ 48-vector]
'''

def Dataset ( model, dratio ) :
    trains = {}
    tests_data = {}
    phones_mapping = {} # {48} to realNumber
    result_mapping = {} # {48} to {39}
    
    training_inputs = []
    training_result = []
    
    # MAPPING 48 to number ( we map 48 to 39 later )
    with open('./MLDS_HW1_RELEASE_v1/phones/48_39.map') as f:    
        i = 0
        for lines in f :
            phones = lines.split('\t')
            phones_mapping[phones[0]] = i
            i += 1
    
    with open('./MLDS_HW1_RELEASE_v1/phones/48_39.map') as f:    
        for lines in f :
            phones = lines.split('\t')
            result_mapping[ phones_mapping[phones[0]] ] = phones[1]
            
# MFCC -------------------------------------------------------------------------------------------------------
    if model == "mfcc":
        # TRAINING X(INPUT)
        with open('./MLDS_HW1_RELEASE_v1/mfcc/train.ark') as f:
            for lines in f :
                frames = lines.split(' ')
                frame2float = [ float(x) for x in frames[1:] ]
                trains[frames[0]] = frame2float
        
        
        # TRAINING Y(OUTPUT)
        with open('./MLDS_HW1_RELEASE_v1/label/train.lab') as f:
            for lines in f :
                labels = lines.split(',')
                training_inputs.append( np.reshape(trains.get(labels[0]), 39 ) )
                training_result.append( vectorized_result(phones_mapping[labels[1].rstrip('\n')] ) )
        
        # 10% for validation
        X_train, X_test, y_train, y_test = cv.train_test_split(training_inputs, training_result, test_size=0.1)        
        y_train = (np.array(y_train)).astype(np.float32)        
        X_train = (np.array(X_train)).astype(np.float32)
        y_test = (np.array(y_test)).astype(np.float32)
        X_test = (np.array(X_test)).astype(np.float32)
          
        # Testing data
        print "Now we process test data"
        with open('./MLDS_HW1_RELEASE_v1/mfcc/test.ark') as f:
            for lines in f :
                frames = lines.split(' ')
                tests_data[frames[0]] = np.reshape([ float(x) for x in frames[1:] ], (39, 1) )        
                
# FBANL -------------------------------------------------------------------------------------------------------
    elif model == "fbank":
        # TRAINING X(INPUT)
        with open('./MLDS_HW1_RELEASE_v1/fbank/train.ark') as f:
            for lines in f :
                frames = lines.split(' ')
                frame2float = [ float(x) for x in frames[1:] ]
                trains[frames[0]] = frame2float
        
        # TRAINING Y(OUTPUT)
        with open('./MLDS_HW1_RELEASE_v1/label/train.lab') as f:
            for lines in f :
                labels = lines.split(',')
                training_inputs.append( np.reshape(trains.get(labels[0]), 69 ) )
                training_result.append( vectorized_result(phones_mapping[labels[1].rstrip('\n')] ) )
        
    
        # 10% for validation
        X_train, X_test, y_train, y_test = cv.train_test_split(training_inputs, training_result, test_size=0.1)
        y_train = (np.array(y_train)).astype(np.float32)
        X_train = (np.array(X_train)).astype(np.float32)
        y_test = (np.array(y_test)).astype(np.float32)
        X_test = (np.array(X_test)).astype(np.float32)
            
        print "Now we process test data"
        with open('./MLDS_HW1_RELEASE_v1/fbank/test.ark') as f:
            for lines in f :
                frames = lines.split(' ')
                tests_data[frames[0]] = np.reshape([ float(x) for x in frames[1:] ], (69, 1) )        

# quadmfcc -------------------------------------------------------------------------------------------------------
    elif model == "quadmfcc":
        # TRAINING X(INPUT)
        monoTrains = {}
        with open('./MLDS_HW1_RELEASE_v1/mfcc/train.ark') as f:
            for lines in f :
                frames = lines.split(' ')
                frame2float = [ float(x) for x in frames[1:] ]
                monoTrains[frames[0]] = frame2float

        zeroFrame = list(repeat(0, 39))
        prevframeL1 = [ float(x) for x in zeroFrame[:] ]
        prevframeL2 = [ float(x) for x in zeroFrame[:] ]
        prevframeR1 = [ float(x) for x in zeroFrame[:] ]
        prevframeR2 = [ float(x) for x in zeroFrame[:] ]
        blankframe  = [ float(x) for x in zeroFrame[:] ]
        # L2 L1 S R1 R2
        frameNameL1 = ""
        frameNameL2 = ""
        frameNameR1 = ""
        frameNameR2 = ""
        
        for sample in monoTrains:
            frames_info = sample.split('_')
            frameNameL2 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-2))
            frameNameL1 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-1))
            frameNameR1 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+1))
            frameNameR2 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+2))           
            if   frames_info[2] == "1":
                trains[sample] = blankframe + blankframe + monoTrains[sample] + monoTrains[frameNameR1] + monoTrains[frameNameR2]
            elif frames_info[2] == "2":
                trains[sample] = blankframe + monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1] + monoTrains[frameNameR2]
            elif monoTrains.get(frameNameR1) is None:
                trains[sample] = monoTrains[frameNameL2] + monoTrains[frameNameL1] + monoTrains[sample] + blankframe + blankframe
            elif monoTrains.get(frameNameR2) is None:
                trains[sample] = monoTrains[frameNameL2] + monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1] + blankframe    
            else:
                trains[sample] = monoTrains[frameNameL2] + monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1] + monoTrains[frameNameR2]    
        monoTrains.clear()
        
        # TRAINING Y(OUTPUT)
        with open('./MLDS_HW1_RELEASE_v1/label/train.lab') as f:
            for lines in f :
                labels = lines.split(',')
                # trains[labels[0]].append(labels[1])
                training_inputs.append( np.reshape(trains.get(labels[0]), (117, 1) ) )
                training_result.append( vectorized_result(phones_mapping[labels[1].rstrip('\n')] ) )
        
        # 10% for validation
        X_train, X_test, y_train, y_test = cv.train_test_split(training_inputs, training_result, test_size=0.1)
        y_train = (np.array(y_train)).astype(np.float32)
        X_train = (np.array(X_train)).astype(np.float32)
        y_test = (np.array(y_test)).astype(np.float32)
        X_test = (np.array(X_test)).astype(np.float32)
            
        # Testing data
        print "Now we process test data"
        with open('./MLDS_HW1_RELEASE_v1/mfcc/test.ark') as f:
            for lines in f :
                frames = lines.split(' ')
                tests_data[frames[0]] = np.reshape([ float(x) for x in frames[1:] ], (39, 1) )   
    

# haxamfcc -------------------------------------------------------------------------------------------------------
    elif model == "hexamfcc": 
        # TRAINING X(INPUT)
        monoTrains = {}
        with open('./MLDS_HW1_RELEASE_v1/mfcc/train.ark') as f:
            for lines in f :
                frames = lines.split(' ')
                frame2float = [ float(x) for x in frames[1:] ]
                # trains[frames[0]] = frames[1:]
                monoTrains[frames[0]] = frame2float

        zeroFrame = list(repeat(0, 39))
        prevframeL1 = [ float(x) for x in zeroFrame[:] ]
        prevframeL2 = [ float(x) for x in zeroFrame[:] ]
        prevframeL3 = [ float(x) for x in zeroFrame[:] ]
        prevframeL4 = [ float(x) for x in zeroFrame[:] ]
        prevframeR1 = [ float(x) for x in zeroFrame[:] ]
        prevframeR2 = [ float(x) for x in zeroFrame[:] ]
        prevframeR3 = [ float(x) for x in zeroFrame[:] ]
        prevframeR4 = [ float(x) for x in zeroFrame[:] ]
        blankframe  = [ float(x) for x in zeroFrame[:] ]
        # L4 L3 L2 L1 S R1 R2 R3 R4
        frameNameL1 = ""
        frameNameL2 = ""
        frameNameL3 = ""
        frameNameL4 = ""
        frameNameR1 = ""
        frameNameR2 = ""
        frameNameR3 = ""
        frameNameR4 = ""
        
        for sample in monoTrains:
            frames_info = sample.split('_')
            frameNameL4 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-4))
            frameNameL3 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-3))
            frameNameL2 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-2))
            frameNameL1 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-1))
            frameNameR1 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+1))
            frameNameR2 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+2))  
            frameNameR3 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+3))
            frameNameR4 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+4))           
            if   frames_info[2] == "1":
                trains[sample] = blankframe + blankframe + blankframe + blankframe + monoTrains[sample] + monoTrains[frameNameR1] + monoTrains[frameNameR2] + monoTrains[frameNameR3] + monoTrains[frameNameR4]
            elif frames_info[2] == "2":
                trains[sample] = blankframe + blankframe + blankframe + monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1] + monoTrains[frameNameR2] + monoTrains[frameNameR3] + monoTrains[frameNameR4]
            elif frames_info[2] == "3":
                trains[sample] = blankframe + blankframe + monoTrains[frameNameL2] + monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1] + monoTrains[frameNameR2] + monoTrains[frameNameR3] + monoTrains[frameNameR4]
            elif frames_info[2] == "4":
                trains[sample] = blankframe + monoTrains[frameNameL3] + monoTrains[frameNameL2] + monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1] + monoTrains[frameNameR2] + monoTrains[frameNameR3] + monoTrains[frameNameR4]
            elif monoTrains.get(frameNameR1) is None:
                trains[sample] = monoTrains[frameNameL4] + monoTrains[frameNameL3] + monoTrains[frameNameL2] + monoTrains[frameNameL1] + monoTrains[sample] + blankframe + blankframe + blankframe + blankframe
            elif monoTrains.get(frameNameR2) is None:
                trains[sample] = monoTrains[frameNameL4] + monoTrains[frameNameL3] + monoTrains[frameNameL2] + monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1] + blankframe + blankframe + blankframe
            elif monoTrains.get(frameNameR3) is None:
                trains[sample] = monoTrains[frameNameL4] + monoTrains[frameNameL3] + monoTrains[frameNameL2] + monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1] + monoTrains[frameNameR2] + blankframe + blankframe
            elif monoTrains.get(frameNameR4) is None:
                trains[sample] = monoTrains[frameNameL4] + monoTrains[frameNameL3] + monoTrains[frameNameL2] + monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1] + monoTrains[frameNameR2] + monoTrains[frameNameR3] + blankframe
            else:
                trains[sample] = monoTrains[frameNameL4] + monoTrains[frameNameL3] + monoTrains[frameNameL2] + monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1] + monoTrains[frameNameR2] + monoTrains[frameNameR3] + monoTrains[frameNameR4] 
        monoTrains.clear()
        
        # TRAINING Y(OUTPUT)
        with open('./MLDS_HW1_RELEASE_v1/label/train.lab') as f:
            for lines in f :
                labels = lines.split(',')
                # trains[labels[0]].append(labels[1])
                training_inputs.append( np.reshape(trains.get(labels[0]), (351) ) )
                training_result.append( vectorized_result(phones_mapping[labels[1].rstrip('\n')] ) )
        trains.clear()
        
        # 10% for validation
        X_train, X_test, y_train, y_test = cv.train_test_split(training_inputs, training_result, test_size=0.1)
        y_train = (np.array(y_train)).astype(np.float32)
        X_train = (np.array(X_train)).astype(np.float32)
        y_test = (np.array(y_test)).astype(np.float32)
        X_test = (np.array(X_test)).astype(np.float32)
        
        # Testing data X
        monoTest = {}
        combineTest = {}
        
        print "Now we process test data"
        with open('./MLDS_HW1_RELEASE_v1/mfcc/test.ark') as f:
            for lines in f :
                frames = lines.split(' ')
                # monoTrains[frames[0]] = np.reshape([ float(x) for x in frames[1:] ], (39, 1) )
                frame2float = [ float(x) for x in frames[1:] ]
                monoTest[frames[0]] = frame2float
        
        zeroFrame = list(repeat(0, 39))
        prevframeL1 = [ float(x) for x in zeroFrame[:] ]
        prevframeL2 = [ float(x) for x in zeroFrame[:] ]
        prevframeL3 = [ float(x) for x in zeroFrame[:] ]
        prevframeL4 = [ float(x) for x in zeroFrame[:] ]
        prevframeR1 = [ float(x) for x in zeroFrame[:] ]
        prevframeR2 = [ float(x) for x in zeroFrame[:] ]
        prevframeR3 = [ float(x) for x in zeroFrame[:] ]
        prevframeR4 = [ float(x) for x in zeroFrame[:] ]
        blankframe  = [ float(x) for x in zeroFrame[:] ]
        # L4 L3 L2 L1 S R1 R2 R3 R4
        frameNameL1 = ""
        frameNameL2 = ""
        frameNameR1 = ""
        frameNameR2 = ""
        frameNameL3 = ""
        frameNameL4 = ""
        frameNameR3 = ""
        frameNameR4 = ""
        
        for sample in monoTest:
            frames_info = sample.split('_')
            frameNameL4 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-4))
            frameNameL3 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-3))
            frameNameL2 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-2))
            frameNameL1 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-1))
            frameNameR1 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+1))
            frameNameR2 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+2))  
            frameNameR3 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+3))
            frameNameR4 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+4))           
            if   frames_info[2] == "1":
                combineTest[sample] = blankframe + blankframe + blankframe + blankframe + monoTest[sample] + monoTest[frameNameR1] + monoTest[frameNameR2] + monoTest[frameNameR3] + monoTest[frameNameR4]
            elif frames_info[2] == "2":
                combineTest[sample] = blankframe + blankframe + blankframe + monoTest[frameNameL1] + monoTest[sample] + monoTest[frameNameR1] + monoTest[frameNameR2] + monoTest[frameNameR3] + monoTest[frameNameR4]
            elif frames_info[2] == "3":
                combineTest[sample] = blankframe + blankframe + monoTest[frameNameL2] + monoTest[frameNameL1] + monoTest[sample] + monoTest[frameNameR1] + monoTest[frameNameR2] + monoTest[frameNameR3] + monoTest[frameNameR4]
            elif frames_info[2] == "4":
                combineTest[sample] = blankframe + monoTest[frameNameL3] + monoTest[frameNameL2] + monoTest[frameNameL1] + monoTest[sample] + monoTest[frameNameR1] + monoTest[frameNameR2] + monoTest[frameNameR3] + monoTest[frameNameR4]
            elif monoTest.get(frameNameR1) is None:
                combineTest[sample] = monoTest[frameNameL4] + monoTest[frameNameL3] + monoTest[frameNameL2] + monoTest[frameNameL1] + monoTest[sample] + blankframe + blankframe + blankframe + blankframe
            elif monoTest.get(frameNameR2) is None:
                combineTest[sample] = monoTest[frameNameL4] + monoTest[frameNameL3] + monoTest[frameNameL2] + monoTest[frameNameL1] + monoTest[sample] + monoTest[frameNameR1] + blankframe + blankframe + blankframe
            elif monoTest.get(frameNameR3) is None:
                combineTest[sample] = monoTest[frameNameL4] + monoTest[frameNameL3] + monoTest[frameNameL2] + monoTest[frameNameL1] + monoTest[sample] + monoTest[frameNameR1] + monoTest[frameNameR2] + blankframe + blankframe
            elif monoTest.get(frameNameR4) is None:
                combineTest[sample] = monoTest[frameNameL4] + monoTest[frameNameL3] + monoTest[frameNameL2] + monoTest[frameNameL1] + monoTest[sample] + monoTest[frameNameR1] + monoTest[frameNameR2] + monoTest[frameNameR3] + blankframe
            else:
                combineTest[sample] = monoTest[frameNameL4] + monoTest[frameNameL3] + monoTest[frameNameL2] + monoTest[frameNameL1] + monoTest[sample] + monoTest[frameNameR1] + monoTest[frameNameR2] + monoTest[frameNameR3] + monoTest[frameNameR4] 
        monoTest.clear()
        
        for keys in combineTest:
            tests_data[keys] = np.reshape(combineTest[keys], (351, 1) )
    
# hexafbank -------------------------------------------------------------------------------------------------------
    elif model == "hexafbank": 
        # TRAINING X(INPUT)
        monoTrains = {}
        with open('./MLDS_HW1_RELEASE_v1/fbank/train.ark') as f:
            for lines in f :
                frames = lines.split(' ')
                frame2float = [ float(x) for x in frames[1:] ]
                monoTrains[frames[0]] = frame2float

        zeroFrame = list(repeat(0, 69))
        prevframeL1 = [ float(x) for x in zeroFrame[:] ]
        prevframeL2 = [ float(x) for x in zeroFrame[:] ]
        prevframeL3 = [ float(x) for x in zeroFrame[:] ]
        prevframeL4 = [ float(x) for x in zeroFrame[:] ]
        prevframeR1 = [ float(x) for x in zeroFrame[:] ]
        prevframeR2 = [ float(x) for x in zeroFrame[:] ]
        prevframeR3 = [ float(x) for x in zeroFrame[:] ]
        prevframeR4 = [ float(x) for x in zeroFrame[:] ]
        blankframe  = [ float(x) for x in zeroFrame[:] ]
        # L4 L3 L2 L1 S R1 R2 R3 R4
        frameNameL1 = ""
        frameNameL2 = ""
        frameNameL3 = ""
        frameNameL4 = ""
        frameNameR1 = ""
        frameNameR2 = ""
        frameNameR3 = ""
        frameNameR4 = ""
        
        for sample in monoTrains:
            frames_info = sample.split('_')
            frameNameL4 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-4))
            frameNameL3 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-3))
            frameNameL2 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-2))
            frameNameL1 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-1))
            frameNameR1 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+1))
            frameNameR2 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+2))  
            frameNameR3 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+3))
            frameNameR4 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+4))           
            if   frames_info[2] == "1":
                trains[sample] = blankframe + blankframe + blankframe + blankframe + monoTrains[sample] + monoTrains[frameNameR1] + monoTrains[frameNameR2] + monoTrains[frameNameR3] + monoTrains[frameNameR4]
            elif frames_info[2] == "2":
                trains[sample] = blankframe + blankframe + blankframe + monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1] + monoTrains[frameNameR2] + monoTrains[frameNameR3] + monoTrains[frameNameR4]
            elif frames_info[2] == "3":
                trains[sample] = blankframe + blankframe + monoTrains[frameNameL2] + monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1] + monoTrains[frameNameR2] + monoTrains[frameNameR3] + monoTrains[frameNameR4]
            elif frames_info[2] == "4":
                trains[sample] = blankframe + monoTrains[frameNameL3] + monoTrains[frameNameL2] + monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1] + monoTrains[frameNameR2] + monoTrains[frameNameR3] + monoTrains[frameNameR4]
            elif monoTrains.get(frameNameR1) is None:
                trains[sample] = monoTrains[frameNameL4] + monoTrains[frameNameL3] + monoTrains[frameNameL2] + monoTrains[frameNameL1] + monoTrains[sample] + blankframe + blankframe + blankframe + blankframe
            elif monoTrains.get(frameNameR2) is None:
                trains[sample] = monoTrains[frameNameL4] + monoTrains[frameNameL3] + monoTrains[frameNameL2] + monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1] + blankframe + blankframe + blankframe
            elif monoTrains.get(frameNameR3) is None:
                trains[sample] = monoTrains[frameNameL4] + monoTrains[frameNameL3] + monoTrains[frameNameL2] + monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1] + monoTrains[frameNameR2] + blankframe + blankframe
            elif monoTrains.get(frameNameR4) is None:
                trains[sample] = monoTrains[frameNameL4] + monoTrains[frameNameL3] + monoTrains[frameNameL2] + monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1] + monoTrains[frameNameR2] + monoTrains[frameNameR3] + blankframe
            else:
                trains[sample] = monoTrains[frameNameL4] + monoTrains[frameNameL3] + monoTrains[frameNameL2] + monoTrains[frameNameL1] + monoTrains[sample] + monoTrains[frameNameR1] + monoTrains[frameNameR2] + monoTrains[frameNameR3] + monoTrains[frameNameR4] 
        monoTrains.clear()
        
        # TRAINING Y(OUTPUT)
        with open('./MLDS_HW1_RELEASE_v1/label/train.lab') as f:
            for lines in f :
                labels = lines.split(',')
                training_inputs.append( np.reshape(trains.get(labels[0]), 621 ) )
                training_result.append( vectorized_result(phones_mapping[labels[1].rstrip('\n')] ) )
        trains.clear()
        
        # 10% for validation
        X_train, X_test, y_train, y_test = cv.train_test_split(training_inputs, training_result, test_size=0.1)
        y_train = (np.array(y_train)).astype(np.float32)
        X_train = (np.array(X_train)).astype(np.float32)
        y_test = (np.array(y_test)).astype(np.float32)
        X_test = (np.array(X_test)).astype(np.float32)
        
        # Testing data X
        monoTest = {}
        combineTest = {}
        
        print "Now we process test data"
        with open('./MLDS_HW1_RELEASE_v1/fbank/test.ark') as f:
            for lines in f :
                frames = lines.split(' ')
                # monoTrains[frames[0]] = np.reshape([ float(x) for x in frames[1:] ], (39, 1) )
                frame2float = [ float(x) for x in frames[1:] ]
                monoTest[frames[0]] = frame2float
        
        zeroFrame = list(repeat(0, 69))
        prevframeL1 = [ float(x) for x in zeroFrame[:] ]
        prevframeL2 = [ float(x) for x in zeroFrame[:] ]
        prevframeL3 = [ float(x) for x in zeroFrame[:] ]
        prevframeL4 = [ float(x) for x in zeroFrame[:] ]
        prevframeR1 = [ float(x) for x in zeroFrame[:] ]
        prevframeR2 = [ float(x) for x in zeroFrame[:] ]
        prevframeR3 = [ float(x) for x in zeroFrame[:] ]
        prevframeR4 = [ float(x) for x in zeroFrame[:] ]
        blankframe  = [ float(x) for x in zeroFrame[:] ]
        # L4 L3 L2 L1 S R1 R2 R3 R4
        frameNameL1 = ""
        frameNameL2 = ""
        frameNameR1 = ""
        frameNameR2 = ""
        frameNameL3 = ""
        frameNameL4 = ""
        frameNameR3 = ""
        frameNameR4 = ""
        
        for sample in monoTest:
            frames_info = sample.split('_')
            frameNameL4 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-4))
            frameNameL3 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-3))
            frameNameL2 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-2))
            frameNameL1 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])-1))
            frameNameR1 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+1))
            frameNameR2 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+2))  
            frameNameR3 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+3))
            frameNameR4 = frames_info[0] + "_" + frames_info[1] + "_" + str( (int(frames_info[2])+4))           
            if   frames_info[2] == "1":
                combineTest[sample] = blankframe + blankframe + blankframe + blankframe + monoTest[sample] + monoTest[frameNameR1] + monoTest[frameNameR2] + monoTest[frameNameR3] + monoTest[frameNameR4]
            elif frames_info[2] == "2":
                combineTest[sample] = blankframe + blankframe + blankframe + monoTest[frameNameL1] + monoTest[sample] + monoTest[frameNameR1] + monoTest[frameNameR2] + monoTest[frameNameR3] + monoTest[frameNameR4]
            elif frames_info[2] == "3":
                combineTest[sample] = blankframe + blankframe + monoTest[frameNameL2] + monoTest[frameNameL1] + monoTest[sample] + monoTest[frameNameR1] + monoTest[frameNameR2] + monoTest[frameNameR3] + monoTest[frameNameR4]
            elif frames_info[2] == "4":
                combineTest[sample] = blankframe + monoTest[frameNameL3] + monoTest[frameNameL2] + monoTest[frameNameL1] + monoTest[sample] + monoTest[frameNameR1] + monoTest[frameNameR2] + monoTest[frameNameR3] + monoTest[frameNameR4]
            elif monoTest.get(frameNameR1) is None:
                combineTest[sample] = monoTest[frameNameL4] + monoTest[frameNameL3] + monoTest[frameNameL2] + monoTest[frameNameL1] + monoTest[sample] + blankframe + blankframe + blankframe + blankframe
            elif monoTest.get(frameNameR2) is None:
                combineTest[sample] = monoTest[frameNameL4] + monoTest[frameNameL3] + monoTest[frameNameL2] + monoTest[frameNameL1] + monoTest[sample] + monoTest[frameNameR1] + blankframe + blankframe + blankframe
            elif monoTest.get(frameNameR3) is None:
                combineTest[sample] = monoTest[frameNameL4] + monoTest[frameNameL3] + monoTest[frameNameL2] + monoTest[frameNameL1] + monoTest[sample] + monoTest[frameNameR1] + monoTest[frameNameR2] + blankframe + blankframe
            elif monoTest.get(frameNameR4) is None:
                combineTest[sample] = monoTest[frameNameL4] + monoTest[frameNameL3] + monoTest[frameNameL2] + monoTest[frameNameL1] + monoTest[sample] + monoTest[frameNameR1] + monoTest[frameNameR2] + monoTest[frameNameR3] + blankframe
            else:
                combineTest[sample] = monoTest[frameNameL4] + monoTest[frameNameL3] + monoTest[frameNameL2] + monoTest[frameNameL1] + monoTest[sample] + monoTest[frameNameR1] + monoTest[frameNameR2] + monoTest[frameNameR3] + monoTest[frameNameR4] 
        monoTest.clear()
        for keys in combineTest:
            tests_data[keys] = np.reshape(combineTest[keys], (621, 1) )
                
    else:
        print "Not implement yet"
    
    return (X_train, X_test, y_train, y_test, tests_data, result_mapping)
                                       
def vectorized_result ( j ) :
    e = np.zeros((48, 1))
    e[j] = 1.0
    return np.reshape( e, 48)


## Model part


In [3]:
srng = RandomStreams(seed=2**30)

# translate data to theano data type
def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

# initialize weight by random
def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

def tanh(X):
    return (1 + T.tanh(X / 2)) / 2

# rectified linear unit
def relu(X):
    # return T.maximum(X, 0.)
    return (X + abs(X)) / 2.

def sigmoid(X):
    return 1.0/(1.0+ T.exp(-X))

# softmax
def softmax(X):
    e_x = T.exp(X - X.max(axis=1).dimshuffle(0, 'x'))
    return e_x / e_x.sum(axis=1).dimshuffle(0, 'x')

def build_shared_zeros(shape):
    """ Builds a theano shared variable filled with a zeros numpy array """
    return theano.shared(value=np.zeros(shape, dtype=theano.config.floatX),
             borrow=True)

# method provided by Hinton
def RMSprop(cost, params, lr=0.001, rho=0.9, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    for p, g in zip(params, grads):
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        g = g / gradient_scaling
        updates.append((acc, acc_new))
        updates.append((p, p - lr * g))
    return updates

# momentum method
def momentum(loss, all_params, update_momentum, update_learning_rate, max_norm):
    all_grads = theano.grad(loss, all_params)
    updates = []

    for param_i, grad_i in zip(all_params, all_grads):
        mparam_i = theano.shared(np.zeros(param_i.get_value().shape,
                                          dtype=theano.config.floatX),
                                 broadcastable=param_i.broadcastable)
        v = update_momentum * mparam_i - update_learning_rate * grad_i
        updates.append((mparam_i, v))
        
        if max_norm != None:
            W = param_i + v
            col_norms = W.norm(2, axis=0)
            desired_norms = T.clip(col_norms, 0, max_norm)
            updates.append( ( param_i , (W * (desired_norms / (1e-6 + col_norms))) ) )
        else:
            updates.append((param_i, param_i + v))

    return updates

def multinominal_cross_entropy(z, X):
    
    L = - T.sum(X * T.log(z) + (1 - X) * T.log(1 - z), axis=1)
    loss = T.sum(L) / X.shape[0]
    
    return loss

def negative_log_likelihood_mean(z, X):
    return -T.mean(T.log(z)[T.arange(X.shape[0]), X])

def negative_log_likelihood_sum(z, X):
    return -T.sum(T.log(z)[T.arange(X.shape[0]), X])


# dropout
# https://github.com/mdenil/dropout/issues/6
# def dropout(X, p=0.):
#     if p > 0. and p < 1.:
#         retain_prob = 1 - p
#         X *= srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
#     return X

def dropout( layer, p=0.):
# http://www.quora.com/How-would-you-implement-drop-out-in-a-deep-neural-network
#     srng = theano.tensor.shared_randomstreams.RandomStreams(
#             rng.randint(999999))
    
    if p > 0. and p < 1.:
        # p=1-p because 1's indicate keep and p is prob of dropping
        mask = srng.binomial(n=1, p=1-p, size=layer.shape)
        # The cast is important because
        # int * float32 = float64 which pulls things off the gpu
        output = layer * T.cast(mask, theano.config.floatX)
        return output
    else:
        return layer

def model(X, para, p_drop_input, p_drop_hidden, relu_bool, train):
    ## assume 6 hidden  layers only
    ## Softmax weight divide by 2??
    for i in range (len(para)/2):
        
        if i is 0:
            ######## Input layer
            if train:
                X = dropout(X, p_drop_input)
            else:
                para[i*2].set_value(para[i*2].get_value()*(1-p_drop_hidden[i-1]))
                
            if relu_bool:
                h = relu(T.dot(X, para[i*2]) +para[i+1])
            else:
                h = sigmoid(T.dot(X, para[i*2]) + para[i+1])

        elif i is 1:
            ######## layer 1
            if train:
                h = dropout(h, p_drop_hidden[i-1])
            else:
                # h *= (1-p_drop_hidden[i-1])
                para[i*2].set_value(para[i*2].get_value()*(1-p_drop_hidden[i-1]))
            
            if (i*2+2) is len(para): ## weight at least have 4, w_h_1, b_1, w_o, b_o
                py_xx = softmax(T.dot(h, para[i*2])+ para[i*2+1])
                break
            
            if relu_bool:
                h2 = relu(T.dot(h, para[i*2] )+ para[i*2+1])
            else: 
                h2 = sigmoid(T.dot(h, para[i*2] ) +para[i*2+1])
        
        elif i is 2:
            ######## layer 2
            if train:
                h2 = dropout(h2, p_drop_hidden[i-1])
            else:
                # h2 *= (1-p_drop_hidden[i-1])
                para[i*2].set_value(para[i*2].get_value()*(1-p_drop_hidden[i-1]))
            
            if (i*2+2) is len(para): ## weight at least have 6, w_h_1, b_1, w_o, b_o
                py_xx = softmax(T.dot(h2, para[i*2])+ para[i*2+1])
                break
            
            if relu_bool:
                h3 = relu(T.dot(h2, para[i*2] )+ para[i*2+1])
            else:
                h3 = sigmoid(T.dot(h2, para[i*2] ) +para[i*2+1])
                
            
        elif i is 3:
            ######## layer 3
            if train:
                h3 = dropout(h3, p_drop_hidden[i-1])
            else:
                # h3 *= (1-p_drop_hidden[i-1])
                para[i*2].set_value(para[i*2].get_value()*(1-p_drop_hidden[i-1]))
            
            if (i*2+2) is len(para): ## weight at least have 8, w_h_1, b_1, w_o, b_o
                py_xx = softmax(T.dot(h3, para[i*2])+ para[i*2+1])
                break
            
            if relu_bool:
                h4 = relu(T.dot(h3, para[i*2] )+ para[i*2+1])
            else:
                h4 = sigmoid(T.dot(h3, para[i*2] ) +para[i*2+1])
                
                
        elif i is 4:
            ######## layer 4
            if train:
                h4 = dropout(h4, p_drop_hidden[i-1])
            else:
                # h4 *= (1-p_drop_hidden[i-1])
                para[i*2].set_value(para[i*2].get_value()*(1-p_drop_hidden[i-1]))
            
            if (i*2+2) is len(para): ## weight at least have 10, w_h_1, b_1, w_o, b_o
                py_xx = softmax(T.dot(h4, para[i*2])+ para[i*2+1])
                break
            
            if relu_bool:
                h5 = relu(T.dot(h4, para[i*2] )+ para[i*2+1])
            else:
                h5 = sigmoid(T.dot(h4, para[i*2] ) +para[i*2+1])
                
                
        elif i is 5:
            ######## layer 5
            if train:
                h5 = dropout(h5, p_drop_hidden[i-1])
            else:
                # h5 *= (1-p_drop_hidden[i-1])
                para[i*2].set_value(para[i*2].get_value()*(1-p_drop_hidden[i-1]))
            
            if (i*2+2) is len(para): ## weight at least have 12, w_h_1, b_1, w_o, b_o
                py_xx = softmax(T.dot(h5, para[i*2])+ para[i*2+1])
                break
            
            if relu_bool:
                h6 = relu(T.dot(h5, para[i*2] )+ para[i*2+1])
            else:
                h6 = sigmoid(T.dot(h5, para[i*2] ) +para[i*2+1])
                
        elif i is 6:
            ######## layer 6
            if train:
                h6 = dropout(h6, p_drop_hidden[i-1])
            else:
                # h6 *= (1-p_drop_hidden[i-1])
                para[i*2].set_value(para[i*2].get_value()*(1-p_drop_hidden[i-1]))
            
            if (i*2+2) is len(para): ## weight at least have 12, w_h_1, b_1, w_o, b_o
                py_xx = softmax(T.dot(h6, para[i*2])+ para[i*2+1])
                break
            
            if relu_bool:
                h7 = relu(T.dot(h6, para[i*2] )+ para[i*2+1])
            else:
                h7 = sigmoid(T.dot(h6, para[i*2] ) +para[i*2+1])
                
    return py_xx
    
    


In [12]:
class DNN():    
    
    def __init__(self, input_shape, activation, hidden_layer,
                 batch, max_epochs, eval_size, output_num_units,
                 drop_input, drop_hidden,
                 patience, up_learning_rate, up_momentum, max_norm):
        
        self.input_shape = input_shape
        self.hidden_layer = hidden_layer
#         self.hidden_layer_1 = hidden_layer_1
#         self.hidden_layer_2 = hidden_layer_2
#         self.hidden_layer_3 = hidden_layer_3
        
        if activation == "relu":
            self.activation = True
        else:
            self.activation = False
        
        self.batch = batch
        self.max_epochs = max_epochs
        self.eval_size = eval_size
        self.output_num_units = output_num_units
        
        self.up_learning_rate = up_learning_rate
        self.up_momentum = up_momentum
        
        self.drop_input = drop_input
        self.drop_hidden = drop_hidden
#         self.drop_hidden_1 = drop_hidden_1
#         self.drop_hidden_2 = drop_hidden_2
#         self.drop_hidden_3 = drop_hidden_3
        
        self.patience = patience
        self.best_valid = -np.inf
        self.best_valid_epoch = 0
        self.best_params = None
        
        self.max_norm = max_norm
        
        self.train_history_ = []

        self.params = []
        for i in range(len(hidden_layer)):
            if i is 0:
                w_h_1 = init_weights((self.input_shape[1], self.hidden_layer[i]))
                b_1 = build_shared_zeros(self.hidden_layer[i])
                self.params.append(w_h_1)
                self.params.append(b_1)
            elif i is 1:
                w_h_2 = init_weights((self.hidden_layer[i-1], self.hidden_layer[i]))
                b_2 = build_shared_zeros(self.hidden_layer[i])
                self.params.append(w_h_2)
                self.params.append(b_2)
            elif i is 2:
                w_h_3 = init_weights((self.hidden_layer[i-1], self.hidden_layer[i]))
                b_3 = build_shared_zeros(self.hidden_layer[i])
                self.params.append(w_h_3)
                self.params.append(b_3)
            elif i is 3:
                w_h_4 = init_weights((self.hidden_layer[i-1], self.hidden_layer[i]))
                b_4 = build_shared_zeros(self.hidden_layer[i])
                self.params.append(w_h_4)
                self.params.append(b_4)
            elif i is 4:
                w_h_5 = init_weights((self.hidden_layer[i-1], self.hidden_layer[i]))
                b_5 = build_shared_zeros(self.hidden_layer[i])
                self.params.append(w_h_5)
                self.params.append(b_5)
            elif i is 5:
                w_h_6 = init_weights((self.hidden_layer[i-1], self.hidden_layer[i]))
                b_6 = build_shared_zeros(self.hidden_layer[i])
                self.params.append(w_h_6)
                self.params.append(b_6)
            elif i is 6:
                w_h_7 = init_weights((self.hidden_layer[i-1], self.hidden_layer[i]))
                b_7 = build_shared_zeros(self.hidden_layer[i])
                self.params.append(w_h_7)
                self.params.append(b_7)
                
            if (i+1) is len(hidden_layer):
                w_o = init_weights((self.hidden_layer[i], self.output_num_units))
                b_o = build_shared_zeros(self.output_num_units)
                self.params.append(w_o)
                self.params.append(b_o)
                break

            
#         w_h_1 = init_weights((self.input_shape[1], self.hidden_layer_1))
#         w_h_2 = init_weights((self.hidden_layer_1, self.hidden_layer_2))
#         w_h_3 = init_weights((self.hidden_layer_2, self.hidden_layer_3))
#         w_h_4 = init_weights((self.hidden_layer_3, self.hidden_layer_4))
#         w_h_5 = init_weights((self.hidden_layer_4, self.hidden_layer_5))
#         w_h_6 = init_weights((self.hidden_layer_5, self.hidden_layer_6))
#         w_o = init_weights((self.hidden_layer_6, self.output_num_units))
        
#         b_1 = build_shared_zeros(self.hidden_layer_1)
#         b_2 = build_shared_zeros(self.hidden_layer_2)
#         b_3 = build_shared_zeros(self.hidden_layer_3)
#         b_4 = build_shared_zeros(self.hidden_layer_4)
#         b_5 = build_shared_zeros(self.hidden_layer_5)
#         b_6 = build_shared_zeros(self.hidden_layer_6)
#         b_o = build_shared_zeros(self.output_num_units)
        
        self.update_learning_rate= theano.shared(floatX( up_learning_rate['start'] ))
        
        ###################### 
        ## need to modify if using dropout
        ######################
        self.update_learning_rate.set_value(0.1)
        ######################
        
        self.lr = np.linspace(up_learning_rate['start'], up_learning_rate['stop'], self.max_epochs)
        # self.lr = np.linspace(up_learning_rate['start'], up_learning_rate['stop'], 20)
        
        self.update_momentum= theano.shared(floatX( up_momentum['start'] ))
        # self.mm = np.linspace(up_momentum['start'], up_momentum['stop'], self.max_epochs)
        self.mm = np.linspace(up_momentum['start'], up_momentum['stop'], up_momentum['epoch'])
    
        X = T.dmatrix()
        Y = T.dmatrix()

        # Construct Theano expression graph
#         py_x_drop = model(X, 
#                      w_h_1,b_1,
#                      w_h_2,b_2,
#                      w_h_3,b_3, 
#                      w_h_4,b_4, 
#                      w_h_5,b_5, 
#                      w_h_6,b_6, 
#                      w_o, b_o,
#                      self.drop_input, 
#                      self.drop_hidden_1, self.drop_hidden_2, self.drop_hidden_3, 
#                      self.drop_hidden_4, self.drop_hidden_5, self.drop_hidden_6,
#                      self.activation, True)
        
#         py_x = model(X, 
#                      w_h_1,b_1,
#                      w_h_2,b_2,
#                      w_h_3,b_3, 
#                      w_h_4,b_4, 
#                      w_h_5,b_5, 
#                      w_h_6,b_6, 
#                      w_o, b_o,
#                      self.drop_input, 
#                      self.drop_hidden_1, self.drop_hidden_2, self.drop_hidden_3, 
#                      self.drop_hidden_4, self.drop_hidden_5, self.drop_hidden_6,
#                      self.activation, False)
        
        py_x_drop = model(X, self.params, 
                     self.drop_input, self.drop_hidden, self.activation, True)
    
        py_x = model(X, self.params, 
                     self.drop_input, self.drop_hidden, self.activation, False)

        y_x = T.argmax(py_x, axis=1)

        # cost = T.mean(T.nnet.categorical_crossentropy(py_x, Y))
        cost = multinominal_cross_entropy(py_x_drop, Y) 
        
#         self.params = [ w_h_1, b_1, 
#                        w_h_2, b_2, 
#                        w_h_3, b_3, 
#                        w_h_4, b_4,
#                        w_h_5, b_5,
#                        w_h_6, b_6,
#                        w_o, b_o]
        
        updates = momentum(cost, self.params, self.update_momentum, self.update_learning_rate, self.max_norm)

        # Compile expressions to functions
        self.train = theano.function(inputs=[X, Y], outputs=[cost], 
                                updates=updates, allow_input_downcast=True, name = "train")
        self.predict = theano.function(inputs=[X], outputs=y_x, allow_input_downcast=True, name = "predict")

        
    def fit(self, x, y):
        X_train, X_test, y_train, y_test = cv.train_test_split(x, y, test_size= self.eval_size)
        yy = np.array(map((lambda x: np.argmax(x)), y_test))
        
#         if any([x.op.__class__.__name__ in ['Gemv', 'CGemv', 'Gemm', 'CGemm'] for x in
#         self.train.maker.fgraph.toposort()]):
#             print 'Used the cpu'
#         elif any([x.op.__class__.__name__ in ['GpuGemm', 'GpuGemv'] for x in
#                   self.train.maker.fgraph.toposort()]):
#             print 'Used the gpu'
#         else:
#             print 'ERROR, not able to tell if theano used the cpu or the gpu'
#             print self.train.maker.fgraph.toposort()
            
        print " "
        print "start training!!!!"
        print " "
            
        epochs = 0
        for i in range(self.max_epochs):
            epochs +=1
            t0 = time()
            
            # To do: modify a function of mini batch and using random
            for start, end in zip(range(0, len(X_train), self.batch), range(self.batch, len(X_train), self.batch)):
                err = self.train(X_train[start:end], y_train[start:end])
            
            score = accuracy_score(yy, self.predict(X_test))
            self.train_history_.append({"epoch":epochs, "err": err, "score":score})
            
            print 'epoch {0} : err = {1}, score = {2}, time ={3} s'.format(epochs, err, score, time() - t0)
            
            if np.isnan(err):
                for qq in range (len(self.params)):
                    self.params[qq].set_value( self.best_params[qq] )
                break
            
            if epochs != 1:
#                 new_lr = floatX(self.lr[epochs - 1])
#                 self.update_learning_rate.set_value(new_lr)
                
                ###################### 
                ## need to modify if using dropout
                ######################
                if epochs >= 3:  # <---- 
                    self.update_learning_rate.set_value(0.01)
                    if epochs > 8:
                        self.update_learning_rate.set_value(0.001)
                        if epochs > 13:
                            self.update_learning_rate.set_value(0.0001)
                    
                if epochs <= self.up_momentum['epoch']:
                    new_mm = floatX(self.mm[epochs - 1])
                    self.update_momentum.set_value(new_mm)
#                     print "momentum update: ", new_mm
                
            current_score = self.train_history_[-1]['score']
            current_epoch = self.train_history_[-1]['epoch']
            if current_score > self.best_valid:
                self.best_valid = current_score
                self.best_valid_epoch = current_epoch
                self.best_params = [w.get_value() for w in self.params]
            elif self.best_valid_epoch + self.patience <= current_epoch:
                print ""
                print "Early stopping."
                print self.best_valid_epoch,self.best_valid
                print "Best valid score {:.6f} at epoch {}.".format(self.best_valid, self.best_valid_epoch)
                
                for qq in range (len(self.params)):
                    self.params[qq].set_value( self.best_params[qq] )
                break


    def prediction(self, x):
        return self.predict(x)

    def outputCSV(self, wfilename, test_data, mapd): # read dictionary for id 
        test_results = []
        
        for xid, xdata in test_data.iteritems():
            test_results.append( (xid, self.predict(xdata.T)) ) 
            
        f = open(wfilename, 'w+')
        f.write("Id,Prediction\n")
        for xid, y in test_results:
            f.write("{0},{1}".format(xid, mapd[y[0]]))
        f.close()
        print "MISSION COMPLETE"
    
    

# Start training!

 ## Data reading

In [10]:
### Data Processing 

data_ratio = 1 # the input we use ( to put more efford on improve parameter)

# X_train, X_test, y_train, y_test, test_data, result_mapping = Dataset("mfcc", data_ratio)
# X_train, X_test, y_train, y_test, test_data, result_mapping = Dataset("fbank", data_ratio)
# trimfcc filled wit buggy :P
# X_train, X_test, y_train, y_test, test_data, result_mapping = Dataset("trimfcc", data_ratio)
# X_train, X_test, y_train, y_test, test_data, result_mapping = Dataset("quadmfcc", data_ratio)
X_train, X_test, y_train, y_test, test_data, result_mapping = Dataset("hexamfcc", data_ratio)
# X_train, X_test, y_train, y_test, test_data, result_mapping = Dataset("hexafbank", data_ratio)



Now we process test data


In [5]:
(np.array(y_train)).shape
# y_train = np.array(y_train)
# ((y_train.flatten())).reshape(len(y_train.flatten())/48,48).astype(np.float32)
# y_train[0].flatten()

(1012340, 48)

In [11]:
(np.array(X_train)).shape

# print X_train[0]
# X_train = np.array(X_train)
# X_train[0].shape
# (X_train.flatten()).reshape(1012340,39)[0]

(1012340, 351)

In [9]:
(np.array(test_data)).shape
print len(test_data)
# len(np.array(test_data.values()[0]))
# np.array(test_data.values()).flatten()

# test_data = np.array(test_data.values())
# np.array(((test_data.flatten()))).reshape(len(test_data.flatten())/39,39).shape


180406


## Train model!

In [13]:
# Test Hexa FBank with no dropout, don't ask what is Hexa it means 4 + 1 + 4
net5_hexaMFCC_dropout_2 = DNN(
    input_shape=(128,351), 
    hidden_layer=[1500,1500,1500,1500 ], # maximum size to 6 layer only
    drop_input=0.2, # usually is 0.2
    drop_hidden=[0.5,0.5,0.5,0.5], # maximum size to 6 layer only, usually use 0.5 
    activation = "relu", # relu, sigmoid
    batch=128, 
    max_epochs=300, 
    eval_size=0.1, 
    output_num_units=48, 
    up_learning_rate = {'start':0.1, 'stop':0.0001}, # make it dynamic increase
    up_momentum = {'start':0.5, 'stop':0.9, 'epoch':20}, # only 40 epochs, and after 40 epochs, use 0.9
    patience=20,
    max_norm = None # need to tune, maybe start from 3 or 4
)


In [ ]:
%time net5_hexaMFCC_dropout_2.fit(X_train, y_train)

In [13]:
# Test Hexa FBank with no dropout, don't ask what is Hexa it means 4 + 1 + 4
net5_hexaMFCC_no_dropout_1 = DNN(
    input_shape=(128,351), 
    hidden_layer=[1500,1500,1500,1500 ], # maximum size to 6 layer only
    drop_input=0, # usually is 0.2
    drop_hidden=[0,0,0,0], # maximum size to 6 layer only, usually use 0.5 
    activation = "relu", # relu, sigmoid
    batch=128, 
    max_epochs=300, 
    eval_size=0.1, 
    output_num_units=48, 
    up_learning_rate = {'start':0.001, 'stop':0.0001}, # make it dynamic increase
    up_momentum = {'start':0.9, 'stop':0.9, 'epoch':20}, # only 40 epochs, and after 40 epochs, use 0.9
    patience=20,
    max_norm = None # need to tune, maybe start from 3 or 4
)


In [ ]:
%time net5_hexaMFCC_no_dropout_1.fit(X_train, y_train)

 
start training!!!!
 
epoch 1 : err = [array(2.029616119147112)], score = 0.572890530849, time =2445.32905817 s
epoch 2 : err = [array(1.7391193694849911)], score = 0.627467056523, time =2451.38642287 s
epoch 3 : err = [array(1.6416497354307606)], score = 0.655678922101, time =2458.45371008 s
epoch 4 : err = [array(1.5524880763519038)], score = 0.67512890926, time =2454.52987194 s
epoch 5 : err = [array(1.4878230589142227)], score = 0.690370824031, time =2444.1128099 s
epoch 6 : err = [array(1.4148098141776795)], score = 0.702106011814, time =2446.80837417 s
epoch 7 : err = [array(1.3519400345133634)], score = 0.71073947488, time =2438.3882761 s
epoch 8 : err = [array(1.2945080705872032)], score = 0.718790129798, time =2442.37712598 s
epoch 9 : err = [array(1.2464762325886007)], score = 0.725487484442, time =2438.101964 s
epoch 10 : err = [array(1.1962067289179408)], score = 0.731868739751, time =2443.24386907 s
epoch 11 : err = [array(1.1819344918102077)], score = 0.736679376494, tim

In [19]:
net5_hexaMFCC_no_dropout_1.train_history_

[{'epoch': 1, 'err': [array(2.029616119147112)], 'score': 0.5728905308493194},
 {'epoch': 2,
  'err': [array(1.7391193694849911)],
  'score': 0.62746705652251222},
 {'epoch': 3,
  'err': [array(1.6416497354307606)],
  'score': 0.65567892210127032},
 {'epoch': 4, 'err': [array(1.5524880763519038)], 'score': 0.6751289092597349},
 {'epoch': 5,
  'err': [array(1.4878230589142227)],
  'score': 0.69037082403145189},
 {'epoch': 6,
  'err': [array(1.4148098141776795)],
  'score': 0.70210601181421262},
 {'epoch': 7,
  'err': [array(1.3519400345133634)],
  'score': 0.71073947487998101},
 {'epoch': 8,
  'err': [array(1.2945080705872032)],
  'score': 0.71879012979828916},
 {'epoch': 9,
  'err': [array(1.2464762325886007)],
  'score': 0.72548748444198585},
 {'epoch': 10,
  'err': [array(1.1962067289179408)],
  'score': 0.73186873975146693},
 {'epoch': 11,
  'err': [array(1.1819344918102077)],
  'score': 0.73667937649406323},
 {'epoch': 12,
  'err': [array(1.1592932717024944)],
  'score': 0.74142086

In [19]:
# Test Hexa FBank with no dropout, don't ask what is Hexa it means 4 + 1 + 4
net5HexaFBank_dropout_2 = DNN(
    input_shape=(128,621), 
    hidden_layer=[1500,1500,1500,1500 ], # maximum size to 6 layer only
    drop_input=0.2, # usually is 0.2
    drop_hidden=[0.5,0.5,0.5,0.5], # maximum size to 6 layer only, usually use 0.5 
    activation = "relu", # relu, sigmoid
    batch=128, 
    max_epochs=300, 
    eval_size=0.1, 
    output_num_units=48, 
    up_learning_rate = {'start':0.1, 'stop':0.0001}, # hinton version of constant lr = 0.1, 
    up_momentum = {'start':0.5, 'stop':0.9, 'epoch':20}, # only 40 epochs, and after 40 epochs, use 0.9
    patience=20,
    max_norm = 4 # need to tune, maybe start from 3 or 4
)


In [20]:
%time net5HexaFBank_dropout_2.fit(X_train, y_train)

 
start training!!!!
 
epoch 1 : err = [array(3.3154811269881197)], score = 0.245569670269, time =2582.24993706 s
epoch 2 : err = [array(3.0614095129058008)], score = 0.316099334216, time =2595.10063887 s
epoch 3 : err = [array(2.9409431049653723)], score = 0.365371317937, time =2563.0335331 s
epoch 4 : err = [array(nan)], score = 0.022373906, time =2560.18848205 s
CPU times: user 3h 41min 31s, sys: 1h 59min 6s, total: 5h 40min 37s
Wall time: 2h 51min 41s


In [11]:
# Test Hexa FBank with no dropout, don't ask what is Hexa it means 4 + 1 + 4
net5HexaFBank_no_dropout_1 = DNN(
    input_shape=(128,621), 
    hidden_layer=[2048,2048,2048,2048 ], # maximum size to 6 layer only
    drop_input=0, # usually is 0.2
    drop_hidden=[0,0,0,0], # maximum size to 6 layer only, usually use 0.5 
    activation = "relu", # relu, sigmoid
    batch=128, 
    max_epochs=300, 
    eval_size=0.1, 
    output_num_units=48, 
    up_learning_rate = {'start':0.001, 'stop':0.0001}, 
    up_momentum = {'start':0.9, 'stop':0.9, 'epoch':30}, # only 40 epochs, and after 40 epochs, use 0.9
    patience=20,
    max_norm = None # need to tune, maybe start from 3 or 4
)


In [12]:
%time net5HexaFBank_no_dropout_1.fit(X_train, y_train)

 
start training!!!!
 
epoch 1 : err = [array(2.5324242669967876)], score = 0.485884189106, time =4534.19684601 s
epoch 2 : err = [array(2.2429373715467134)], score = 0.546031965545, time =4524.72641611 s


KeyboardInterrupt: 

## Save model!!

In [ ]:
import sys
sys.setrecursionlimit(10000)

# import cPickle as pickle
# with open('net5_hexaMFCC_no_dropout_1.pickle', 'wb') as f:
#     pickle.dump(net5_hexaMFCC_no_dropout_1, f, -1)y_test

import cPickle as pickle
with open('net5_hexaMFCC_no_dropout_1_Amazing.pickle', 'wb') as f:
    pickle.dump(net5_hexaMFCC_no_dropout_1, f, -1)


## Load model

In [26]:
import cPickle as pickle

with open('net5-HAXAKILL2458748.pickle', 'rb') as f:
    net5 = pickle.load(f)


## validation result

In [18]:
y_pred = net5_hexaMFCC_no_dropout_1.prediction(X_test)

yyy = np.array(map((lambda x: np.argmax(x)), y_test))

print metrics.classification_report((yyy), (y_pred))
print accuracy_score(yyy, y_pred)


             precision    recall  f1-score   support

          0       0.82      0.62      0.70      2447
          1       0.71      0.83      0.77      2999
          2       0.62      0.59      0.60      1622
          3       0.79      0.72      0.75      2270
          4       0.79      0.64      0.71      1126
          5       0.56      0.58      0.57      1794
          6       0.79      0.78      0.79      2746
          7       0.78      0.75      0.76      1089
          8       0.69      0.68      0.68       761
          9       0.83      0.73      0.78      6607
         10       0.73      0.61      0.67      1207
         11       0.71      0.69      0.70      1463
         12       0.77      0.74      0.75       791
         13       0.61      0.67      0.64      2590
         14       0.74      0.61      0.67      1002
         15       0.61      0.54      0.58       561
         16       0.58      0.42      0.49       386
         17       0.83      0.79      0.81   

# write predict file

In [ ]:
net5_hexaMFCC_no_dropout_1.outputCSV("net5_hexaMFCC_no_dropout_1.csv", test_data, result_mapping)

In [18]:
type(test_data)
len(test_data)

180406